In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np 
from time import sleep
from pyfirmata import Arduino, SERVO

# import supervision as sv



class VideoProcessor:
    def __init__(self, source_video) -> None: 
        self.source_video = source_video
        # self.model YOLO("yolo-Weights/diet_engine_all_best_4.pt")
        self.model YOLO("models/Final_model.pt") 
        # self.box_annotator = sv. BoxAnnotator()
        self.board Arduino ("COM8")
        self.pin = 9
        self.board.digital [self.pin].mode SERVO
        self.INITIALANGLE = 60
        self.board.digital [self.pin].write(self.INITIALANGLE)


    def show_video(self):
        cap = cv2.VideoCapture(self.source_video)
        while cap.isOpened():
            ret, frame cap.read()
        if not ret:
            break
        #frame = cv2.flip(frame, 1)
        frame, name, center_point = self.process_frame (frame)
        print(name, center_point)
        if name == 'Bad':
            cv2.putText(frame, "Faulty", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        else:
            cv2.putText(frame, "Not Faulty", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2) 
        cv2.imshow("frame", frame)


        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
        cap.release()
        cv2.destroyAllWindows()

    def predict(self, frame: np.ndarray) -> np.ndarray: 
        return self.model.predict(frame)[0]


    def process_frame (self, frame) -> np.ndarray:
        
        result = self.predict(frame)
        class_name = None
        center_point = None
        for box in result.boxes:
            food size None
            x1, y1, x2, y2 [round(x) for x in box.xyxy[0].tolist()]
            center_point = (x1+x2) // 2, (y1 y2) // 2
            class_id box.cls[0].item()
            probability round(box.conf[0].item(), 2)
            if probability < 0.3:
                continue
                
            class_name = result.names[class_id]
            #Get the dimensions of the image
            height, width, frame.shape
            
            #Calculate the position for the horizontal line
            line_y int (height 0.6)

            #Draw a horizontal line at the bottom 90% of the image 
            cv2.line (frame, (0, line y), (width, line_y), (0, 255, 0), 2)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            if probability > 0.3: 
                cv2.putText(frame, f" (class_name) (probability)", (x1, y110), cv2.FONT HERSHEY SIMPLEX, 0.5, (255, 0, 0), 2) 
                if class name == "Bad" and ((y1 < line y < y2) or (y1 < line_y <= y2)):
                    sleep(0.1)
                    self.unlock()
                    sleep(0.1)
                    self.lock()

        return frame, class name, center_point

    def unlock(self):
        self.board.digital[self.pin].write(130)
        sleep(1)
    def lock(self): 
        self.board.digital[self.pin].write(68) 
        sleep(1)
        
if __name__=="__main__":
    source_video2
    processor Video Processor (source_video)
    processor.show_video()